In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import json

In [2]:
import dataset
import algorithm

In [3]:
CONFIG = json.load(open('config.json'))

In [4]:
Dataset = dataset.download_dataset(CONFIG['data']['filename'])

In [5]:
Dataset.head()

,area,material,condicion,anio_construccion,anio_remodelacion,sotano,calefaccion,aire_acondicionado,area_construida_piso_1,area_construida_piso_2,...,banios,banios_sin_ducha,dormitorios,cocinas,chimeneas,area_garage,area_piscina,mes_venta,anio_venta,precio
0,8450,7,5,2003,2003,Gd,GasA,Y,856,854,...,2,1,3,1,0,548,0,2,2008,208500
1,9600,6,8,1976,1976,Gd,GasA,Y,1262,0,...,2,0,3,1,1,460,0,5,2007,181500
2,11250,7,5,2001,2002,Gd,GasA,Y,920,866,...,2,1,3,1,1,608,0,9,2008,223500
3,9550,7,5,1915,1970,TA,GasA,Y,961,756,...,1,0,3,1,1,642,0,2,2006,140000
4,14260,8,5,2000,2000,Gd,GasA,Y,1145,1053,...,2,1,4,1,1,836,0,12,2008,250000


In [6]:
data = dataset.get_data(Dataset)

In [7]:
normalize_data = dataset.Normalizer.transform(data)

In [8]:
data_x, data_y = dataset.split_data_x_y(normalize_data)

In [9]:
data_x_train, data_x_test = dataset.split_data_train_test(data_x, CONFIG["data"]["train_split_size"])
data_y_train, data_y_test = dataset.split_data_train_test(data_y, CONFIG["data"]["train_split_size"])

In [10]:
dataset_train = dataset.TimeSeriesDataset(data_x_train, data_y_train)
dataset_test = dataset.TimeSeriesDataset(data_x_test, data_y_test)

In [11]:
train_dataloader = DataLoader(dataset_train, CONFIG["training"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(dataset_test, CONFIG["training"]["batch_size"], shuffle=True)

In [12]:
model = algorithm.NNModel(config_model=CONFIG["model"], config_training=CONFIG["training"])

In [13]:
for epoch in range(CONFIG["training"]["num_epoch"]):
    loss_train, lr_train = model.run_epoch(train_dataloader, is_training=True)
    loss_test, lr_test = model.run_epoch(test_dataloader)
    if epoch == 0 or ((epoch+1)%10 == 0):
        print('Epoch[{}/{}] | loss train:{:.6f}, test:{:.6f} | lr:{:.6f}'
              .format(epoch+1, CONFIG["training"]["num_epoch"], loss_train, loss_test, lr_train))

Epoch[1/100] | loss train:0.006030, test:0.001155 | lr:0.010000
Epoch[10/100] | loss train:0.001037, test:0.000379 | lr:0.010000
Epoch[20/100] | loss train:0.000985, test:0.000425 | lr:0.010000
Epoch[30/100] | loss train:0.000517, test:0.000430 | lr:0.010000
Epoch[40/100] | loss train:0.000472, test:0.000526 | lr:0.010000
Epoch[50/100] | loss train:0.000406, test:0.000330 | lr:0.010000
Epoch[60/100] | loss train:0.000483, test:0.000305 | lr:0.010000
Epoch[70/100] | loss train:0.000454, test:0.000483 | lr:0.010000
Epoch[80/100] | loss train:0.000308, test:0.000267 | lr:0.010000
Epoch[90/100] | loss train:0.000358, test:0.000269 | lr:0.010000
Epoch[100/100] | loss train:0.000360, test:0.000334 | lr:0.010000


In [14]:
for idx, (x, y) in enumerate(test_dataloader):
    x = x.to(CONFIG["training"]["device"])
    out = model(x)
    out_vector = out.detach().numpy()
    y_vector = y.numpy()
    
    for i in range(len(y_vector)):
        price_predict = dataset.Normalizer.inverse_transform(out_vector[i])
        price_real = dataset.Normalizer.inverse_transform(y_vector[i])
        print('Price predict: {0}, Price real: {1}, Diferencia: {2}'.format(price_predict, price_real, price_predict-price_real))
        
    break

Price predict: 377826, Price real: 381000, Diferencia: -3174
Price predict: 142553, Price real: 148000, Diferencia: -5447
Price predict: 164178, Price real: 177000, Diferencia: -12822
Price predict: 93932, Price real: 119000, Diferencia: -25068
Price predict: 186686, Price real: 203000, Diferencia: -16314
Price predict: 306802, Price real: 262000, Diferencia: 44802
Price predict: 116800, Price real: 130500, Diferencia: -13700
Price predict: 172932, Price real: 186700, Diferencia: -13768
Price predict: 116673, Price real: 134500, Diferencia: -17827
Price predict: 138629, Price real: 120000, Diferencia: 18629
Price predict: 270055, Price real: 392500, Diferencia: -122445
Price predict: 123683, Price real: 139000, Diferencia: -15317
Price predict: 147986, Price real: 149300, Diferencia: -1314
Price predict: 195843, Price real: 207000, Diferencia: -11157
Price predict: 118148, Price real: 105000, Diferencia: 13148
Price predict: 134182, Price real: 180500, Diferencia: -46318
Price predict: